In [1]:
import numpy as np
import random
from math import log, sqrt

def prod(factors):
    total = 1
    for i in factors:
        total *= i
    return total

In [17]:
def bits2vec(bits):
    n = len(bits)
    if bits.count('1') + bits.count('0') != len(bits):
        raise ValueError("bad bits!")
    vec = np.array([0 for i in range(2**n)])
    index = 0
    for i, bit in enumerate(bits):
        index += int(bit) * 2**(n-i-1)
    vec[index] = 1
    return vec

def vec2bits(vec):
    cvalid(vec)
    n = int(log(len(vec), 2))
    index = list(vec).index(1)
    return bin(index)[2:].zfill(n)
    
def cvalid(bit):
    length = len(bit)
    n = log(length, 2)
    if n != int(n):
        raise ValueError("vector size not a power of 2!")
    count0 = 0
    count1 = 0
    for elt in bit:
        if elt == 0:
            count0 += 1
        if elt == 1:
            count1 += 1
    if count1 != 1:
        raise ValueError("more than one bit set to 1!")
    if count0 != len(bit) - 1:
        raise ValueError("some bits set to something other than 0/1!")

def cproduct(*bits):
    for bit in bits:
        cvalid(bit)
    n = len(bits)
    product = np.array([0 for i in range(2**n)])
    # find index of the ones:
    indices = []
    product_index = 0
    for i, bit in enumerate(bits):
        index = bit.index(1)
        product_index += index * 2**(n-i-1)
    product[product_index] = 1
    return product
    
def cnot(a):
    matrix = np.array([[0, 1],
                       [1, 0]])
    return matrix.dot(a)

def cand(a, b):
    product = cproduct(a, b)
    matrix = np.array([[1, 1, 1, 0],
                       [0, 0, 0, 1]])
    return matrix.dot(product)

def cor(a, b):
    product = cproduct(a, b)
    matrix = np.array([[1, 0, 0, 0],
                       [0, 1, 1, 1]])
    return matrix.dot(product)

def cxor(a, b):
    product = cproduct(a, b)
    matrix = np.array([[1, 0, 0, 1],
                       [0, 1, 1, 0]])
    return matrix.dot(product)

bit0 = bits2vec('0')
bit1 = bits2vec('1')

In [18]:
def pvalid(bit):
    length = len(bit)
    n = log(length, 2)
    if n != int(n):
        raise ValueError("vector size not a power of 2!")
    total = 0
    for part in bit:
        if part < 0:
            raise ValueError("negative bit value!")
        total += part
    if abs(total - 1) > 1e-6:
        raise ValueError("bits don't sum to 1!")

def pproduct(*bits):
    for bit in bits:
        pvalid(bit)
    n = len(bits)
    product = np.array([0.0 for i in range(2**n)])
    for i in range(2**n):
        binary = [int(j) for j in bin(i)[2:].zfill(n)]
        product[i] = prod(bits[j][binary[j]] for j in range(n))
    return product

def psample(a):
    pvalid(a)
    length = len(a)
    index = 0
    cumulative = 0
    r = random.random()
    for i in range(length):
        cumulative += a[i]
        if cumulative >= r:
            sample = np.array([0 for j in range(length)])
            sample[i] = 1
            return sample

In [19]:
def qvalid(qbit):
    length = len(qbit)
    n = log(length, 2)
    if n != int(n):
        raise ValueError("vector size not a power of 2!")
    total = 0
    for part in qbit:
        total += abs(part)**2
    if abs(total - 1) > 1e-6:
        raise ValueError("squared magnitudes don't sum to 1!")

def qproduct(*bits):
    for bit in bits:
        qvalid(bit)
    n = len(bits)
    product = np.array([0.0 for i in range(2**n)])
    for i in range(2**n):
        binary = [int(j) for j in bin(i)[2:].zfill(n)]
        product[i] = prod(bits[j][binary[j]] for j in range(n))
    return product

def qsample(a):
    qvalid(a)
    length = len(a)
    index = 0
    cumulative = 0
    r = random.random()
    for i in range(length):
        cumulative += abs(a[i])**2
        if cumulative >= r:
            sample = np.array([0 for j in range(length)])
            sample[i] = 1
            return sample
        
def qhadamard(a):
    matrix = np.array([[1, 1],
                       [1, -1]])
    matrix = 1/sqrt(2) * matrix
    return matrix.dot(a)